In [2]:
import numpy as np
import os
import math
import glob
import h5py
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [3]:
# ---------- FILES & DATASETS ------------------------------------------------------------------------------------------
folder_path = "C:/Users/txiki/OneDrive/Documents/Studies/MSc_Geomatics/2Y/Thesis/Images/Examples"
file_path = "C:/Users/txiki/OneDrive/Documents/Studies/MSc_Geomatics/2Y/Thesis/Images/Examples/ECO_01-05-2024(00.59).h5"
datasets = ['SDS/LST', 'SDS/Emis1', 'SDS/Emis2', 'SDS/Emis3', 'SDS/Emis4', 'SDS/Emis5', 'SDS/QC']

def explore_h5_group(name, obj):
    # group or dataset
    if isinstance(obj, h5py.Group):
        print(f"📂 Group: {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"📄 Dataset: {name} | Shape: {obj.shape} | Data Type: {obj.dtype}")


with h5py.File(file_path, 'r') as f:
    print(f"📖 Exploring HDF5 File: {file_path}")
    f.visititems(explore_h5_group)

📖 Exploring HDF5 File: C:/Users/txiki/OneDrive/Documents/Studies/MSc_Geomatics/2Y/Thesis/Images/Examples/ECO_01-05-2024(00.59).h5
📂 Group: L2 LSTE Metadata
📄 Dataset: L2 LSTE Metadata/AncillaryNWP | Shape: () | Data Type: object
📄 Dataset: L2 LSTE Metadata/BandSpecification | Shape: (6,) | Data Type: float32
📄 Dataset: L2 LSTE Metadata/CloudMaxTemperature | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/CloudMeanTemperature | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/CloudMinTemperature | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/CloudSDevTemperature | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/Emis1GoodAvg | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/Emis2GoodAvg | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/Emis3GoodAvg | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/Emis4GoodAvg | Shape: (1,) | Data Type: float64
📄 Dataset: L2 LSTE Metadata/Emis5GoodAvg | Shape: (1

In [5]:
# ---------- AOI -------------------------------------------------------------------------------------------------------
# area of interest (same area for all images)
SW_lat, SW_lon = 38.69758, -3.91992
NE_lat, NE_lon = 38.85585, -3.72656
check_pixels = [(38.75012, -3.87543), (38.80567, -3.81234)]


def get_aoi(file, sw_lat, sw_lon, ne_lat, ne_lon):
    with h5py.File(file, 'r') as f:
        # bbox from metadata
        north = f['StandardMetadata/NorthBoundingCoordinate'][()]
        south = f['StandardMetadata/SouthBoundingCoordinate'][()]
        east = f['StandardMetadata/EastBoundingCoordinate'][()]
        west = f['StandardMetadata/WestBoundingCoordinate'][()]

        rows, cols = f[datasets[0]].shape

        # lat/lon grids
        lat_grid = np.linspace(north, south, rows)[:, None]  # column
        lon_grid = np.linspace(west, east, cols)[None, :]  # row vector

        # masking to get pixels from area of interest
        lat_mask = (lat_grid >= sw_lat) & (lat_grid <= ne_lat)
        lon_mask = (lon_grid >= sw_lon) & (lon_grid <= ne_lon)
        valid_rows, valid_cols = np.where(lat_mask)[0], np.where(lon_mask)[1]

        if len(valid_rows) == 0 or len(valid_cols) == 0:
            print(f"Warning: AOI not found in {os.path.basename(file)}")
            return None
        return valid_rows.min(), valid_rows.max(), valid_cols.min(), valid_cols.max(), lat_grid, lon_grid

'''
def get_pixel_index(lat, lon, lat_grid, lon_grid):
    row_idx = np.abs(lat_grid - lat).argmin()  # closest row
    col_idx = np.abs(lon_grid - lon).argmin()  # closest column
    return row_idx, col_idx
'''


'\ndef get_pixel_index(lat, lon, lat_grid, lon_grid):\n    row_idx = np.abs(lat_grid - lat).argmin()  # closest row\n    col_idx = np.abs(lon_grid - lon).argmin()  # closest column\n    return row_idx, col_idx\n'

In [7]:
# -------------------- CLOUDS and QC ------------------------------------------------------------------------------------
# checking pixel quality
def check_pixel_quality(qc_value):
    # bits extracted according to ECOSTRESS manual guide
    bit1 = (qc_value >> 0) & 1  # Bit 1
    bit2 = (qc_value >> 1) & 1  # Bit 2

    if bit1 == 0 and bit2 == 0:
        return 1.0  # Best Quality
    elif bit1 == 1 and bit2 == 0:
        return 0.75  # Nominal Quality
    elif bit1 == 0 and bit2 == 1:
        return 0.25  # Cloud Detected
    else:
        return 0.0  # Missing/Bad Data

def aoi_cloud_mask(folder_path):
    files = glob.glob(os.path.join(folder_path, "*.h5"))
    images = {dataset: [] for dataset in datasets}
    weights = {dataset: [] for dataset in datasets}
    for file in files:
        indices = get_aoi(file, SW_lat, SW_lon, NE_lat, NE_lon)
        if indices is None:
            continue
        row_min, row_max, col_min, col_max, lat_grid, lon_grid = indices

        with h5py.File(file, 'r') as f:
            # QC for quality checking
            qc_data = f["SDS/QC"][:]
            # aoi from QC
            qc_aoi = qc_data[row_min:row_max + 1, col_min:col_max + 1]

            # cloud mask for pixels detected as clouds (10)
            cloud_mask = ((qc_aoi >> 0) & 1 == 0) & ((qc_aoi >> 1) & 1 == 1)

            pixel_weights = np.vectorize(check_pixel_quality)(qc_aoi)

            for dataset_name in datasets:
                data = f[dataset_name][:].astype(float)

                scale_factor = f[dataset_name].attrs.get("scale_factor", [1.0])[0]
                add_offset = f[dataset_name].attrs.get("add_offset", [0.0])[0]
                fill_value = f[dataset_name].attrs.get("_FillValue", [None])[0]

                # scaling + conditions
                data = data * scale_factor + add_offset
                # invalid pixels
                if fill_value is not None:
                    data[data == fill_value] = np.nan
                if 'LST' in dataset_name:
                    data -= 273.15  # K to ºC

                # crop the area of interest
                cropped_data = data[row_min:row_max + 1, col_min:col_max + 1]
                cropped_data[cloud_mask] = np.nan  # remove clouds but keep pixels as nan

                weight_data = pixel_weights.copy()
                weight_data[np.isnan(cropped_data)] = 0.25

                images[dataset_name].append(cropped_data)
                weights[dataset_name].append(weight_data)

    return images, weights


In [8]:
# --------------------------------- DISPLAY ----------------------------------------------------------------------------------
def display(images):
    fig, axes = plt.subplots(1, len(datasets), figsize=(8 * len(datasets), 8))
    if len(datasets) == 1:
        axes = [axes]

    for ax, (dataset_name, data_list) in zip(axes, images.items()):
        if data_list:
            avg_data = np.nanmean(np.stack(data_list), axis=0)
            im = ax.imshow(avg_data, cmap='gray', extent=[SW_lon, NE_lon, SW_lat, NE_lat])
            ax.set_title(dataset_name, fontsize=14)
            ax.set_xlabel("Longitude", fontsize=12)
            ax.set_ylabel("Latitude", fontsize=12)
            fig.colorbar(im, ax=ax, orientation='vertical', fraction=0.05, pad=0.06)
        plt.tight_layout()
        plt.show()
        '''
                if 'Emis' in dataset_name or 'LST' in dataset_name:
                    print(f"\n{dataset_name}:")
                    for lat, lon in check_pixels:
                        row_idx, col_idx = get_pixel_index(lat, lon, lat_grid, lon_grid)
                        if row_min <= row_idx <= row_max and col_min <= col_idx <= col_max:
                            value = data[row_idx, col_idx]
                            unit = "°C" if 'LST' in dataset_name else ""
                            print(f"  - ({lat}, {lon}): {value:.2f} {unit}")

                            # quality display
                            if qc_data is not None:
                                qc_value = qc_data[row_idx, col_idx]
                                quality = check_pixel_quality(qc_value)
                                print(f"    -> QC Flag: {qc_value} ({quality})")
                            else:
                                print("    -> No QC data available")

                        else:
                            print(f"  - ({lat}, {lon}): Out of AOI bounds")

            plt.tight_layout()
            plt.show()
'''
# extract_and_plot_aoi(folder_path)

In [9]:
# -------------- NORMALIZATION -----------------------------------------------------------------------------------------
# values to range [0, 1]
def normalize(data):
    min_val = np.min(data)
    max_val = np.max(data)
    return ((data - min_val) / (max_val - min_val + 1e-8)).astype(np.float32)


In [11]:
# -------------- PRE-PROCESSING ----------------------------------------------------------------------------------------

# Images  into Numpy arrays
def arrays(images):
    data = []
    for i in range(len(next(iter(images.values())))):
        stacked_images = []
        for dataset_images in images.values():
            stacked_images.append(normalize(dataset_images[i]))
        stacked_images = np.stack(stacked_images, axis=-1)
        data.append(stacked_images)
        # turns all images into a single numpy array

    return np.array(data)  # returning (nº samples, 256, 256, nº channels)


'''Shape after arrays:
· 1 image - (256, 256, 1)
· 10 images - (256, 256, 10)
· 150 images - (256, 256, 150)'''


'Shape after arrays:\n· 1 image - (256, 256, 1)\n· 10 images - (256, 256, 10)\n· 150 images - (256, 256, 150)'

In [12]:
# -------------- PATCHING ----------------------------------------------------------------------------------------------
def patching(images, patch_size=256, stride=256, label=None):     # stride=128 means 50% overlapping
    """
    Extracts non-overlapping patches from the h5 images.
    :param image: 2D NumPy array (grayscale) or 3D NumPy array (multi-channel)
    :param patch_size: Size of patches
    :param stride: same as patch size to ensure no overlapping
    :return: List of patches
    """
    patches = []
    h, w, c = images.shape  # height and width

    for i in range(0, h - patch_size + 1, stride):  # height
        for j in range(0, w - patch_size + 1, stride):  # width
            patch = images[i:i+patch_size, j:j+patch_size, :]
            patches.append(patch)

    return np.array(patches)

'''
Shape after patching:
· 1 image - (nº patches, 256, 256, 1)
· 10 images - (nº patches, 256, 256, 10)
· 150 images - (nº patches, 256, 256, 150)
'''


'\nShape after patching:\n· 1 image - (nº patches, 256, 256, 1)\n· 10 images - (nº patches, 256, 256, 10)\n· 150 images - (nº patches, 256, 256, 150)\n'

In [13]:
# -------------- SPLITS ------------------------------------------------------------------------------------------------
def splitting(patches, labels, test_size=0.1, val_size=0.2, random_state=42):
    # Split data into training (80%), validation (10%), and test (10%)
    # First into training and temporal data (validation + testing)
    X_train, X_temp, y_train, y_temp = train_test_split(patches, labels, test_size=test_size, random_state=random_state)
    # Then into validation and testing
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=val_size/(1-test_size), random_state=random_state)

    return X_train, X_val, X_test, y_train, y_val, y_test


In [14]:
# -------------- WORKFLOW ----------------------------------------------------------------------------------------------

aoi_images, pixel_weights = aoi_cloud_mask(folder_path)
display(aoi_images)

stacked = arrays(aoi_images)
print(stacked.shape)  # ensure num_channels == 6 (1 LST + 5 Emissivity bands)
print(stacked[..., 0])  # ensure LST is expected data
print(stacked[..., 1:])  # ensure Emis is expected data
patches = np.concatenate([patching(img) for img in stacked], axis=0)
weight_patches = np.concatenate([patching(img) for img in pixel_weights], axis=0)
print(f"Total patches from all images: {patches.shape}")

labels = np.zeros((patches.shape[0], 256, 256, 1))  # change when I have labels ready
X_train, X_val, X_test, y_train, y_val, y_test = splitting(patches, labels)
# checks to make sure CNN receives correct input shape
print(X_train.shape, X_val.shape, X_test.shape)  # expected (N, 256, 256, nº channels)
print(y_train.shape, y_val.shape, y_test.shape)  # expected (N, 256, 256, 1)
# in case shape is not correct, use below
'''
X_train = np.expand_dims(X_train, axis=-1)  # Makes it (N, 256, 256, 1)
X_val = np.expand_dims(X_val, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)
'''

np.save("X_train.npy", X_train)
np.save("X_val.npy", X_val)
np.save("X_test.npy", X_test)
np.save("y_train.npy", y_train)
np.save("y_val.npy", y_val)
np.save("y_test.npy", y_test)
np.save("pixel_weights.npy", weight_patches)


print("✅ Data saved successfully!")

MemoryError: Unable to allocate 232. MiB for an array with shape (5632, 5400) and data type float64